-   [Exotica](#chap:cXX)
    -   [Introduction](#introduction)
    -   [Chaos](#sec:cha)
    -   [Local Lyapunov exponents](#local-lyapunov-exponents)
    -   [Coexisting attractors](#coexisting-attractors)
    -   [Repellors / Unstable manifolds](#sec:c10rep)
    -   [Invasion orbits](#invasion-orbits)
    -   [Stochastic resonance](#sec:c10stocres)
    -   [Predictability: Empirical dynamic modelling](#sec:c10emd)
    -   [Appendix: Making a pomp-simulator](#sec:pomp)

Exotica
=======

Introduction
------------

Chapter \[chap:c10\] discussed how a linear approximation to the perennially nonlinear dynamics of infectious disease can provide important insights on invasion, stability and resonant periodicity. As remarked by more generally, linear approximation can often provide remarkably useful insights for nonlinear ecological systems as long as they are not *too* nonlinear.

From a dynamical system’s point of view, dynamics are (approximately) linear if the system does not ‘miss-behave’ as it approach (diverge) from its stable (unstable) fix-points. Thus, while the simple SIR model is mathematically-speaking nonlinear (because of the $\beta S I /N$ term), its dynamics can be thought of as being ‘linear’ because of its smooth inwards spiraling towards the endemic equilibrium (the stable focus) and logistic divergence from the disease-free equilibrium (the unstable node) when $R_0>1$. However, highly infectious immunizing diseases has the potential for exhibiting dynamics so non-linear that crazy things can happen; Things that requires a different set of tools. The chaotic fluctuations seen in the seasonally forced SEIR model and measles in prevaccination US (section \[sec:c4bif\]) are one such highly nonlinear phenomenon. There are however other dynamic ‘exotica’ that can arise when stochasticity and nonlinearity interacts, or when there are great perturbations (such as introduction of mass vaccination) to the nonlinear epidemic clockworks. The following sections explores this and discuss some useful tools.

Chaos
-----

In nonlinear systems, a perturbation will either dampen (‘dissipate’) or expand as it interacts with the dynamic clockwork. The hallmark of a chaotic attractor is ‘sensitive dependence on initial conditions’[1]: Two very nearby trajectories will diverge exponentially over time. The classic way to quantify this is through the dominant Lyapunov exponent: $$\lambda = \lim_{T \rightarrow \infty} \frac{1}{T} \log(\prod_t^T J_t U_0), 
\label{eq:LE}$$ where $J_t$ is the Jacobian evaluated on the point of the attractor at time $t$ and for a 2D system like the TSIR $U_0$ is the unit vector $\{1,0\}$. A chaotic attractor has $\lambda > 0$. The Jacobian of the TSIR model is: $$J_t =
 \begin{bmatrix}
   1 - \beta_s I_t^\alpha/N &
  - \beta_s S_t (I_t^{\alpha - 1}  \alpha)/N\\
   \beta_s I_t^\alpha/N  &
   \beta_s S_t (I_t^{\alpha - 1} \alpha)/N
\end{bmatrix}.$$ We can estimate the Lyapunov exponent numerically by simulating the TSIR a long time . We consider the measles dynamics in New York between 1920 and 1941 (fig. \[fig:cXXny\]) . We first fit the parameters using the protocol discussed in chapter \[chap:c6\]; The profile likelihood on $\overline{S}$ suggests a mean fraction of susceptibles of 0.051.

In [ ]:
data(dalziel)
NY = na.omit(dalziel[dalziel$loc=="NEW YORK",])
NY = NY[NY$year %in% c(1920:1940),]
plot(NY$decimalYear, sqrt(NY$cases), type = "b", 
     xlab = "Year", ylab = "Sqrt(cases)")
#Susceptible reconstruction and 
#correcting for underreporting
cum.reg = smooth.spline(cumsum(NY$rec), 
     cumsum(NY$cases), df = 5)
D = - resid(cum.reg) #The residuals
rr = predict(cum.reg, deriv = 1)$y
Ic = NY$cases/rr
Dc = D/rr 
#Align lagged variables
seas = rep(1:26, 21)[1:545]
lInew = log(Ic[2:546])
lIold = log(Ic[1:545])
Dold = Dc[1:545]
#TSIR fit
N = NY$pop
offsetN = -log(N[1:545])
lSold = log(0.051*N[1:545] + Dold)
glmfit = glm(lInew ~ -1 +as.factor(seas) + lIold +
     offset(lSold+offsetN))

We use the `SimTsir2`-function from chapter \[chap:c6\] to simulate a 200 year long deterministic trajectory from the fitted model. The result is a highly erratic trajectory in the phase plane (fig. \[fig:nyattr\]).

In [ ]:
sim2 = SimTsir2(beta = exp(glmfit$coef[1:26]), alpha = 0.98,
     B = rep(median(NY$rec), 5200), N = median(N), 
     inits = list(Snull = exp(lSold[1]), Inull = Ic[1]), 
     type = "det")
Sattr = sim2$S[2601:5200]
Iattr = sim2$I[2601:5200]
plot(Sattr, Iattr, log = "y", type = "l",
     xlab = "S", ylab = "I")
points(sim2$S[seq(2601, 5200, by = 26)], 
     sim2$I[seq(2601, 5200, by = 26)],pch = 19, col = "red")
legend("bottomright", c("Trajectory", "Strobe"), 
     pch = c(NA, 19), lty = c(1, NA) , col = c("black","red"))

Calculating the Lyapunov exponent is a bit involved, so we write a function to do it. Because we will be wanting to study the attractor in greater detail, we make the function both calculate the Lyapunov exponent, but also store all the Jacobian elements evaluated along the attractor.

In [ ]:
TSIRlyap=function(I, S, alpha, bt, N){
   IT = length(I)
   s = length(bt)
   j11 = rep(NA, IT);  j12 = rep(NA, IT)
   j21 = rep(NA, IT);  j22 = rep(NA, IT)
    #initial unit vector
    J = matrix(c(1,0),ncol=1)
    #loop over the attractor
    for(i in 1:IT) {
         j11[i] = 1 -  bt[((i - 1) %% s) + 1] * 
            I[i]^alpha/N
         j12[i] = -( bt[((i - 1) %% s) + 1] * S[i] * 
            (I[i]^(alpha - 1) * alpha)/N)
         j21[i] =  bt[((i - 1) %% s) + 1] * I[i]^alpha/N
         j22[i] =  bt[((i - 1) %% s) + 1] * S[i] * 
            (I[i]^(alpha - 1) * alpha)/N
    
         J = matrix(c(j11[i],j12[i],j21[i],j22[i]), 
            ncol = 2, byrow = TRUE)%*%J
    }
   res = list(lyap = log(norm(J))/IT, j11 = j11, j12 = j12, 
         j21 = j21, j22 = j22, I = I, S = S, alpha = alpha, 
         bt = bt, N = N)
   class(res) = "lyap"
   return(res)
}

We apply the function to the last 100 years of the simulated dynamics:

In [ ]:
nylyap = TSIRlyap(I = Iattr, S = Sattr, alpha = 0.98, 
     bt = exp(glmfit$coef[1:26]), N = median(N))
nylyap$lyap

The exponent is positive indicating that the deterministic skeleton of the TSIR model for measles in New York is a chaotic attractor as concluded by . This contrasts with the negative Lyapunov exponent of measles in London testifying to the stability of its biennial limit cycle (see below).

Local Lyapunov exponents
------------------------

 suggested that it is useful to study the *local* Lyapunov exponents to understand short term predictability, and also how noise and nonlinearity will interact in epidemic systems. The idea is that regardless of whether dynamics is asymptotically stable, cyclic or chaotic, there is likely to be regions in the phase plane with expansion in which stochastic divergence will be amplified and regions of contraction were perturbations will be dampened. used local Lyapunov exponents to understand the remarkable predictability of prevaccination measles in London. Local Lyapunov exponents are similar in nature to the global exponent, except that rather than evaluating a product of Jacobians across the attractor, the Jacobians are evaluated locally. Armed with an object produced with the `TSIRlyap`-function it is easy to write a second function to calculate local exponents across `m`-steps along the attractor (or anywhere else in the phase plane, such as along a ‘repellor’; see section \[sec:c10rep\]). Since the TSIR is a discrete time model, contraction occurs if the largest eigenvalue of the stability matrix is inside the unit circle – thus $\log(| \lambda |)<0$ is the cut-off between contraction and expasion. The `TSIRllyap`-function calculates the Local Lyapunov exponents for outputs from the `TSIRlyap`-function. The parameter `m` controls the number of iterations along the attractor on which to calculate the product .

In [ ]:
TSIRllyap = function(x, m = 1){
     llyap = rep(NA, length(x$I))
     for(i in 1:(length(x$I)-m)){
         J = matrix(c(1,0,0,1), ncol = 2)
         for(k in 0:(m-1)){
             J  =  matrix(c(x$j11[(i+k)], x$j12[(i+k)], 
                x$j21[(i+k)], x$j22[(i+k)]), ncol  =  2, 
                byrow = TRUE)%*%J}
         llyap[i] = log(max(abs(eigen(J)$values)))/m
}
res = list(llyap = llyap, I = x$I, S = x$S)
class(res) = "llyap"
return(res)
}

For ease of use we can also write a function to visualize the local exponents:

In [ ]:
plot.llyap = function(x, inches = .5){
   pm = x$llyap>0
   plot(NA, xlim = range(x$S), ylim = range(x$I), xlab = "S", 
        ylab = "I", log = "y")
   symbols(x$S[pm], x$I[pm], circles = x$llyap[pm], 
        inches = inches, add = TRUE)
   symbols(x$S[!pm],x$I[!pm], squares = -x$llyap[!pm], 
        inches = inches, add = TRUE, bg = 2)
}

We can study the measles NY attractor using the local Lyaponov exponents. Despite the fact that the overall attractor is chaotic, we see distinct areas of contraction associated with the collapse of epidemics and post-epidemic troughs (fig. \[fig:nylls\]).

In [ ]:
nyllyap = TSIRllyap(nylyap, m = 5)
plot.llyap(nyllyap, inches = 0.15)

Section \[sec:c6est\] provided TSIR estimates for prevaccination measles in London and showed the limit-cycle nature of the dynamics. Biweekly transmission estimates were:

In [ ]:
beta = c(27.71, 43.14, 37.81, 33.69, 31.64, 32.10, 30.16, 
 24.68, 30.19, 31.53, 30.31, 26.02, 26.57, 25.68, 23.21, 
 19.21, 17.50, 20.50, 29.92, 35.85, 32.65, 28.34, 31.11, 
 29.89, 26.89, 39.38)

Median biweekly birth rate for London was 2083. We can simulate the London attractor and associated global and local Lyapunov exponents.

In [ ]:
sim=SimTsir2(beta=beta, alpha=0.98, 
B=rep(2083, 5200), N=3.3E6, inits=list(Snull=133894, 
     Inull=474))
Sattr=sim$S[5149:5200]
Iattr=sim$I[5149:5200]
lonlyap=TSIRlyap(I=Iattr, S=Sattr, alpha=0.98, 
     bt=beta, N=3.3E6)
lonlyap$lyap
lonllyap=TSIRllyap(lonlyap, m=1)

The dominant Lyapunov exponent is negative testifying to the stability of the limit cycles. We can look in greater detail across the biennial attractor (fig. \[fig:cXXlond\]). Interestingly there is potential for significant divergence during the growth phase of the minor and major epidemics, however, the post-epidemic convergence is apparently strong enough to overcome this to result in a strongly dissipative cyclic attractor.

In [ ]:
pm=lonllyap$llyap>0
plot(NA, xlim=c(1,52), ylim=range(Iattr), 
     xlab="Biweek", ylab="I", log="y")
symbols((1:52)[pm],Iattr[pm], circles=
     lonllyap$llyap[pm], inches=.3, add=TRUE)
symbols((1:52)[!pm],Iattr[!pm], squares=
     -lonllyap$llyap[!pm], inches=.3, add=TRUE, bg=2)

To visualize this effect more clearly we can plot the long-term deterministic attractor with 20 stochastic simulation (assuming demographic stochasticity only)(Fig. \[fig:cXXlond\]). The simulations show that despite abundant variability – particularly during the minor epidemics – the trajectories exhibits long term predictability; That is except for the rare stochastic trajectory that escapes onto the opposite-year coexisting attractor towards the end of the simulations. discuss how the area around Norwich locked on to the opposite-year coexisting attractor compared to the rest of England and Wales for about 15 years following World War II.

In [ ]:
sim = SimTsir2(beta = beta, alpha = 0.98, B = rep(2083, 520), 
   N = 3.3E6, inits = list(Snull = 133894, Inull = 474), 
   type = "det")
plot(sqrt(sim$I), ylab = "Sqrt(Cases)", xlab = "Biweek")
for(i in 1:20){
   sim = SimTsir2(beta = beta, alpha = 0.98, B = rep(2083, 520), 
       N = 3.3E6, inits = list(Snull = 133894, Inull = 474),
       type = "stoc")
lines(sqrt(sim$I))}
sim = SimTsir2(beta = beta, alpha = 0.98, B = rep(2083, 520), 
   N = 3.3E6, inits = list(Snull = 133894, Inull = 474), 
   type = "det")
points(sqrt(sim$I), col = 2)

During the first biweek of 1940, 23 cases of measles were reported in New York City. Given our estimate of the reporting rate of $22.54\%$ in that biweek, a best guess of the incidence is 102; Correspondingly, the best guess of the number of susceptibles is $402,153$. To visualize the ‘sensitive dependence on initial conditions’ of the chaotic New York measles attractor we can forward simulate 10 years of dynamics, assuming there were either 5 more (less) infecteds (susceptibles) or 5 less (more) infecteds (susceptibles). The rapid deterministic divergence (fig. \[fig:cXXdetsim\]) is a stark contrast to the predictability of the London attractor (fig. \[fig:cXXlond\]).

In [ ]:
sim2 = SimTsir2(beta = exp(glmfit$coef[1:26]), alpha = 0.98, 
    B = rep(median(NY$rec), 260), N = median(N), 
    inits = list(Snull = 402153, Inull = 102))
sim3 = SimTsir2(beta = exp(glmfit$coef[1:26]), alpha = 0.98, 
    B = rep(median(NY$rec), 260), N = median(N), 
    inits = list(Snull = 402153-5, Inull = 102+5))
sim4 = SimTsir2(beta = exp(glmfit$coef[1:26]), alpha = 0.98, 
    B = rep(median(NY$rec), 260), N = median(N), 
    inits = list(Snull = 402153+5, Inull = 102-5))
plot(sim2$I[1:260], type = "l", ylab = "I", 
    xlab = "Biweek (from 1940)")
lines(sim3$I[1:260], col = 2)
lines(sim4$I[1:260], col = 3)

Coexisting attractors
---------------------

Another nonlinear complication is how seasonally-forced epidemic systems can exhibit coexisting attractors as, for example, seen in the seasonally forced SEIR model (section \[sec:c5rec\]). Stochastic perturbation can push dynamics between different basins of attraction leading to erratic dynamics not predicted by basic theory.

One of the many puzzles about whooping cough dynamics is the apparent contradiction between historical herd immunity and historical multi-annual epidemics *versus* current circulation in adults[2]. To reconcile these seemingly mutually exclusive facets of whooping cough epidemiology, proposed an hypothesis that immunity to whooping cough wanes over time but re-exposure can boost immunity. This lead to the following SIRWS compartmental model were W is a waning class that is under the influence of two competing processes: return to the $S$ class at a rate of $2 \omega$ or boost back to the $R$ class at a rate proportional to the force of infection: $$\begin{aligned}
\frac{dS}{dt} & =& \mu (1-p) N  - \mu S  - \beta S I / N + 2 \omega W\\
   \label{eq:sirws}
   \frac{dI}{dt} & =& \beta S I / N - (\mu + \gamma) I\\
   \frac{dR}{dt} & = &\gamma I - (\mu - 2 \omega) R +  \kappa \beta S W / N +  \mu p R\\
   \frac{dW}{dt} & = & 2 \omega R - \kappa \beta W I / N - (2 \omega + \mu) W
   \label{eq:sirww}\end{aligned}$$ In the absence of boosting, immunity is expected to last for a mean of $1/\omega$ year (and distributed according to a Gamma-distribution with a shape-parameter of 2; see section \[sec:c2adv\]). The parameter $\kappa$ scales how sensitive boosting is relative to infection, and $p$ is the fraction of children vaccinated at birth. A surprising finding is that in parts of the parameter space, a limit cycle coexists with a fix point attractor. We can use the forwards and backwards bifurcation algorithm of section \[sec:c5rec\] to look at this. We first define the gradients (using the log-trick):

In [ ]:
sirwmod = function(t, logy, parms){
  y = exp(logy)
   S = y[1]
   I = y[2]
   R = y[3]
   W = y[4]
   with(as.list(parms),{
   dS = mu * (1-p) * N  - mu * S  - beta * S * I / N + 
      2*omega * W
   dI = beta * S * I / N - (mu + gamma) * I
   dR = gamma * I - mu * R - 2*omega * R +  
      kappa * beta * W * I / N + mu*p*N
   dW = 2*omega * R - kappa * beta * W * I / N - 
      (2*omega +mu)* W 
   res = c(dS/S, dI/I, dR/R, dW/W)
   list(res)
 })
 }

We assume susceptible recruitment is reduced by vaccination and bifurcate on this parameter (fig. \[fig:sirwsbif\]). The bifurcation analysis reveals the coexistence of a fix-point and a cyclic attractor when vaccination is in the 20-40% range.

In [ ]:
require(deSolve)
start = log(c(S = 0.06, I = 0.01, R = 0.92, W  =  0.01))
res = matrix(NA, ncol = 100, nrow = 5000)
p = seq(0.01, 1, length = 100)
#Forwards 
for(i in 1:100){
  times = seq(0, 200, by = 0.01)
  paras  = c(mu = 1/70, p=p[i], N = 1, beta = 200, 
     omega = 1/10, gamma = 17, kappa = 30)
  out = as.data.frame(ode(start, times, sirwmod, paras))
  start = c(S = out[20001,2], I = out[20001,3], 
     R = out[20001,4], W = out[20001,5])
  res[,i] = out$I[15002:20001]
  cat(i, "\r")
}
#Backwards
res2 = matrix(NA, ncol = 100, nrow = 5000)
start = c(S = -1.8, I = -5.8, R = 1.9, W = -1.9)
for(i in 100:1){
  paras  = c(mu = 1/70, p=p[i], N = 1, beta = 200, 
     omega = 1/10, gamma = 17, kappa = 30)
  out = as.data.frame(ode(start, times, sirwmod, paras))
start = c(S = out[20001,2], I = out[20001,3], 
     R = out[20001,4], W = out[20001,5])
res2[,i] = out$I[15002:20001]
cat(i, "\r")
}
plot(NA, xlim = range(p), ylim = range(res), 
   ylab = "Log(I)", xlab = "p")
for(i in 1:100) points(rep(p[i], 2), range(res[,i]))
for(i in 1:100) points(rep(p[i], 2), 
   range(res2[,i]), col = 2)

Figure \[fig:sirws\] shows trajectories towards the two attractors assuming $20\%$ vaccination but with different initial conditions. For the given parameters the limit cycle is stable and has period of 1.8 years and the fix-point attractor is a stable focus with a damping period of 1.2 years. Using a seasonally-forced version of the SIRWS model, explored the hypothesis that the regime-shifts in prevaccination whooping cough dynamics in Copenhagen (fig. \[fig:cXXwhoop\]) was due to stochastic switching between a low-amplitude noisy annual attractor and a high amplitude cyclic attractor. In the end, the best evidence suggests that the major recurrent outbreaks between 1915 and 1925 was instead a ‘fly-by’ of an unstable multiannual ‘almost attractor’ .

In [ ]:
paras  = c(mu = 1/70, p = 0.2, N = 1, beta = 200, 
     omega = 1/10, gamma = 17, kappa=30)
start = c(S = -1, I = -5, R = 3.3, W = 0)
times = seq(0, 30, by = 1/52)
out = as.data.frame(ode(start, times, sirwmod, paras))
plot(out$time, exp(out$I), xlab = "Year", 
     ylab = "I", type = "l", ylim = c(0,0.05))
start = c(S = -1.8, I = -5.8, R = 1.9, W = -1.9)
times = seq(0, 30, by = 1/52)
out = as.data.frame(ode(start, times, sirwmod, paras))
lines(out$time, exp(out$I), col = 2)

Repellors / Unstable manifolds
------------------------------

 studied a seasonally forced SEIR model (section \[sec:c4seas\]) of chickenpox (assuming that shedding from zoster can be ignored); They assumed a latent period and infectious period of around 10 days and sinusoidally-forced transmission with a $\beta_0$ of 537/year, $\beta_1 = 0.3$ and a birth rate of 0.02.

The integrated ODEs predicts robust annual epidemics in the presence of seasonal forcing (fig. \[fig:cXXchkdet\]).

In [ ]:
times = seq(0, 100, by = 1/120)
start = c(S = 0.06, E = 0, I = 0.001, R = 0.939)
cparas  = c(mu = 0.02, N = 1, beta0 = 537, beta1 = 0.3, 
     sigma = 36, gamma = 34.3)
out = as.data.frame(ode(start, times,seirmod2, cparas))
plot(out$time, out$I, type = "l", xlab = "Year", ylab = 
     "Prevalence", xlim = c(91,100), ylim = c(0, 0.0015))

In contrast, stochastic simulations of the model (see appendix section \[sec:pomp\] for detail) – assuming stochasticity in the seasonal transmission rate – exhibit dynamics with conspicuous ‘regime-shifts’; Periods with the expected somewhat variable annual outbreaks are interspersed with periods of violent multiannual cycles with a period of around 4 years (fig. \[fig:cXXchkstoc2\]) that is completely unrelated to the damping period of the unforced SEIR model ($\sim 2.8$ years for these parameters). This appears to be a dynamical phenomenon different to what we have studied previously.

For the stochastic simulations we build a pomp object using the ‘C-snippets’ detailed in the appendix \[sec:pomp\]. The `dat`-data object defines the times for the stochastic simulation. We are not working with data so the `reports` column is just a dummy.

In [ ]:
dat = data.frame(time = seq(0, 500, by = 1/52), reports = NA) 
seirp = pomp(dat, times = "time",t0 = 0,
    rprocess = euler.sim(Csnippet(rproc),delta.t = 1/52/20),
    skeleton = vectorfield(Csnippet(skel)),
    rmeasure = Csnippet(robs),
    dmeasure = Csnippet(dobs),
    initializer = Csnippet(rinit),
    params = c(iota = 0,beta0 = 537,beta1 = 0.3,sigma = 36,
         gamma = 34.3,alpha = 0.015,rho = 0.6,theta = 1,
         b = 0.02,mu = 0.02,pop0 = 5e8,
         S0 = 0.06,E0 = 0,I0 = 0.001,R0 = 0.939),
         paramnames = c("iota","beta0","beta1","gamma",
         "sigma","alpha","rho","theta", "b","mu","pop0",
         "S0","E0","I0","R0"),
    statenames = c("S","E","I","R","inc","pop"),
    zeronames = "inc")

We simulate deterministic and stochastic trajectories to produce figure \[fig:cXXchkdet\] and \[fig:cXXchkstoc2\].

In [ ]:
detsim = trajectory(seirp, as.data.frame = TRUE)
plot(detsim$time, detsim$I/5E8, type = "l", 
     xlim = c(101, 110), xlab = "year", ylab = "prevalence")

The annual stoboscopic section of the deterministic and stochastic simulation is shown in figure \[fig:cXXchkstoc2\]:

In [ ]:
par(mfrow = c(1,2))
stocsim = simulate(seirp, seed = 3495135, 
     as.data.frame = TRUE, nsim = 1)
plot(stocsim$time, stocsim$I/5E8, type = "l", xlim = c(150, 
     250), xlab = "Year", ylab = "Prevalence")
sel = seq(105, length(stocsim$I), by = 52)
plot(stocsim$S[sel]/5E8, stocsim$I[sel]/5E8, 
     log = "xy", xlab = "S", ylab = "I")
sel2 = sel[401:500]
points(detsim$S[sel2]/5E8, detsim$I[sel2]/5E8, col = 2, 
     pch = 21, bg = 2)

studied the apparent similarity of the stochastic trajectory in the S-I phase plane (fig. \[fig:cXXchkstoc2\]) to the quasiperiodic chaotic attractor of the parametrically-nearby model of section \[sec:c4bif\] (fig. \[fig:c4poincr\]). They stipulated that the stochastic dynamics of the deterministically annual system is intermittently governed by the weakly unstable ‘ghost’ of the nearby 4-year quasi-periodic chaotic attractor. To study this further we turn to the notion of ‘invasion orbits’.

Invasion orbits
---------------

Studying highly nonlinear, stochastic dynamical systems is complicated by the intermingling of two different sources of dynamic variability: the variability due to deterministic instability and the variability due to stochastic forcing . In order to elucidate this complexity, we may think of the stochastic forcing as a perturbation to the nonlinear system which laws subsequently will attempt to return the system to the deterministic attractor. In section \[sec:c4seas\], we discussed how to study the long-term asymptotic behavior of the seasonally-forced SEIR system through numeric integration from arbitrary initial conditions and discarding the initial transient dynamics. ‘Invasion orbits’ is the flip-side of this; Systematically distribute initial conditions across the phase plane and numerically integrate the transients to describe the trajectories toward the deterministic attractor. For ‘linear systems’ or approximately linear systems – in the dynamical systems sense – the invasion orbits will be monotonic trajectories towards a stable node and smooth spirals towards a stable focus (see fig. \[fig:siplane\]); The period of the inward spiral will be determined by the dampening period of the focus as discussed in chapter \[chap:c10\]. For highly nonlinear systems, in contrast, the approach towards the attractor may not be smooth. We can illustrate this using chickenpox SEIR model. We first have to set up a systematic grid of initial conditions:

In [ ]:
starts = expand.grid(S = seq(0.02, 0.1, length = 10), 
     E = seq(1E-8, 0.0125, length = 10), 
     I = seq(1E-8, 0.005, length = 10))
starts$R = 1-apply(starts,1,sum)

For each of these 1000 initial conditions we will simulate the system for 100 years and then store the annual pointcare section in the S-I plane. suggested this be done after discarding a short burn-in period (we use 5 years):

In [ ]:
#times for integration
itimes  = seq(0, 100, by = 1/52)
#points for stroboscopic section
isel = seq(1, length(itimes), by = 52)
#list to fill with results
cporbs = list(S = matrix(NA, ncol = dim(starts)[1], 
     nrow = length(isel)), I = matrix(NA, 
     ncol = dim(starts)[1], nrow = length(isel)))

We now integrate to obtain the 1000 invasion orbits:

In [ ]:
for (i in 1:dim(starts)[1]){
   out2b = as.data.frame(ode(as.numeric(starts[i,]), 
      itimes, seirmod2, cparas))
   cporbs$S[,i] = out2b[,2][isel]
   cporbs$I[,i] = out2b[,4][isel]
}

We can finally plot the stroboscopic section of the invasion orbits in the S-I phase plane with the deterministic attractor superimposed (fig \[fig:cXXpointcare\]) to reveal that the stochastic simulation is largely governed by the unstable highly nonlinear structure in the phase plane dubbed variously a ‘repellor’, a ‘chaotic saddle’ or an ‘unstable manifold’. also referred to it as an ‘almost attractor’.

In [ ]:
#Invasion orbits
plot(as.vector(cporbs$S[-c(1:5),]), 
   as.vector(cporbs$I[-c(1:5),]), pch=20, cex=0.25, 
   log="xy", ylab="I", xlab="S")
#Stochastic simulation
sel=seq(105, length(stocsim$I), by=52)
points(stocsim$S[sel]/5E8, stocsim$I[sel]/5E8, col=2)
#Deterministic attractor
times  = seq(0, 1000, by=1/120)
start = c(S=0.06, E=0, I=0.001, R = 0.939)
out = as.data.frame(ode(start, times, seirmod2, cparas))
sel=seq(120*100, length(times), by=120)
points(out$S[sel], out$I[sel], pch=21, col="white", 
     bg="white")

discussed how the multiannual cycles in whooping cough following vaccination may be explained as the dynamics chasing a periodic saddle (a periodic ‘almost attractor’).

Stochastic resonance
--------------------

Whooping cough in pre-vaccination Copenhagen generally exhibited low amplitude ‘fuzzy’ epidemics, with the exception of a 10 year period of violent epidemics starting around 1915 (fig. \[fig:cXXwhoop\]). We use wavelet analysis with the ‘crazy climber’ ridge finding algorithm to characterize the transitions in dynamics.

In [ ]:
data(pertcop)
require(Rwave)
#Wavelet decompostion
no = 8
nv = 16
a = 2^seq(1, no+1-1/nv, by = 1/nv)
wfit = cwt(sqrt(pertcop$cases), no, nv, plot = FALSE)
wspec  =  Mod(wfit)
#Crazy climber
crcinc<-crc(wspec, nbclimb = 10, bstep = 100)
fcrcinc<-cfamily(crcinc, ptile = 0.5, nbchain = 1000, 
     bstep = 10)

used the ratio of variation in the multiannual *versus* high-frequency part of the wavelet spectrum as a simple measure of the time varying ‘signal-to-noise’ (SNR) ratio in the whooping cough dynamics:

In [ ]:
sigind = which((a/52)>3 & (a/52)<4)
noiseind = which((a/52)<0.5)
snr = apply(wspec[, sigind], 1, 
     sum)/apply(wspec[, noiseind], 1, sum)

We can finally make a composite plot of incidence, wavelet spectrum and signal-to-noise ratio (fig. \[fig:cXXwhoop\]) to highlight how the major epidemics are much less ‘noisy’ than the low amplitude cycles. fit a seasonally-forced SIRWS (eq. \[eq:sirws\]) model to the data and concluded that the curious run of violent epidemics of whooping cough appeared to trace out an unstable multiannual ‘almost attractor’.

In [ ]:
par(mfrow = c(3,1), mar = c(0,4,2,1))
layout(matrix(c(1,1,2,2,2,3), ncol = 1))
#Top panel
plot(as.Date(pertcop$date), pertcop$cases, xlab = "",
   ylab = "Sqrt(cases)", type = "l", bty = 'l', 
   xlim = c(as.Date("1901-01-01"), 
   as.Date("1938-01-01")), xaxt='n', yaxt='n')
axis(2, at = seq(0,200,by = 100), labels = FALSE)
axis(2, at = seq(50,250,by = 100), labels = TRUE)
#Mid panel
par(mar = c(0,4,0.25,1))
image(x = as.Date(pertcop$date, origin = "1900-01-07"), 
  wspec, col = gray((30:10)/32), y = a/52, ylim = c(0,5), 
  ylab = "Period (year)", main = "", xaxt = "n", yaxt = "n")
contour(x = as.Date(pertcop$date, origin = "1900-01-07"), 
   wspec, y = a/52, ylim = c(0,5), 
   zlim = c(quantile(wspec)[4], max(wspec)), add = TRUE)
axis(2, at = 0:4) 
ridges<-fcrcinc[[1]]
ridges[which(ridges<1.5*10^-5)]<-NA
image(x = as.Date(pertcop$date, origin = "1900-01-07"), 
   y = a/52, z = ridges, add = TRUE, col = "black")
#Bottom panel
par(mar = c(3,4,0.25,1), tcl = -0.4)
plot(x = as.Date(pertcop$date, origin = "1900-01-07"), snr, 
   type = "l", bty = "l",xaxt = "n", yaxt = "n", ylab = "SNR")
axis.Date(1, at = seq(as.Date("1900-01-01"), 
  as.Date("1938-01-01"), "years")) 

In addition to highlighting the potential influence of unstable manifolds in disease dynamics, pre-vaccination Copenhagen whooping cough hints at another *exotic* feature of certain nonlinear dynamical systems: increased stochasticity can sometimes *increase* predictability through a process of ‘stochastic resonance’ . We can illustrate this phenomenon with the seasonally-forced stochastic SEIR model introduced in section \[sec:c10rep\]. The pomp `C-snippets` are detailed in section \[sec:pomp\]. We simulate 500 years of weekly data across a range of 126 transmisssion variances between 0 and 0.025 (given by the `alpha`-vector). The stochastic dynamics is prone to extinction so, for each parameter set, we change the pseudorandom `seed` until a 500-year persistent time series is produced (using the `while`-loop):

In [ ]:
dat = data.frame(time = seq(0, 500, by = 1/52), reports = NA) 
sds = rep(NA, 126)
alpha = seq(0,0.025, by = 0.0002)
Smat = Imat = matrix(NA, nrow = 26001, ncol = 126)
for(i in 1:126){
  seirp = pomp(dat, times = "time", t0 = 0,
    rprocess = euler.sim(Csnippet(rproc), delta.t = 1/52/20),
    skeleton = vectorfield(Csnippet(skel)),
    rmeasure = Csnippet(robs),
    dmeasure = Csnippet(dobs),
    initializer = Csnippet(rinit),
    params = c(iota = 0, beta0 = 537, beta1 = 0.3, sigma = 36,
       gamma = 34.3, alpha = alpha[i], rho = 0.6, theta = 1,
       b = 0.02, mu = 0.02, pop0 = 5e8,
       S0 = 0.06, E0 = 0, I0 = 0.001, R0 = 0.939),
       paramnames = c("iota", "beta0", "beta1", "gamma",
       "sigma", "alpha", "rho", "theta", "b", "mu", "pop0",
       "S0", "E0" ,"I0", "R0"),
    statenames = c("S","E","I","R","inc","pop"),
    zeronames = "inc"
    )
  stocsim$I[26001] = 0
  j = -1
  while(stocsim$I[26001]==0){
  j = j+1
  stocsim = simulate(seirp, seed = 3495131+j, 
       as.data.frame = TRUE, nsim = 1)
  sds[i] = 3495131+j
  }
  Imat[,i] = stocsim$I
  Smat[,i] = stocsim$S
}

To study *stochastic resonance* we simulate the model across a range of stochastic variability in the transmission rate. We use wavelet analysis to quantify ‘predictability’ as a function of stochasticity (fig \[fig:cXXstocres\]).

In [ ]:
predn = rep(NA, 126)
#Set the number of "octaves" and "voices"
no = 8; nv = 10
#then calculate the corresponding periods
a = 2^seq(1,no+1-1/nv, by = 1/nv)
sel2 =  a> = 39 & a <260 #Multiannual signal
sel = a<39 #High frequency noise
for(i in 1:126){
     wfit = cwt(sqrt(Imat[,i]), no, nv, plot = FALSE)
     wspec = Mod(wfit)
     predn[i] = sum(wspec[,sel2])/sum(wspec[sel])
}
plot(alpha, predn, xlab = "Noise variance", 
     ylab = "'Predictability'")

The wavelet ‘signal-to-noise’-ratio indicates the curious phenomenon that ‘predictability’ increases with noise up to a point then decays (fig. \[fig:cXXstocres\]). The effect comes about because with low noise variance, the system rarely interacts with the high-amplitude quasi-periodic ‘almost attractor’ and at high noise variance the stochasticity breaks the epidemiological clockwork. Stochasticity can push a dynamical system towards an ‘almost stable’ multiannual cycle, as argued was the case of pre-vaccination whooping cough in Copenhagen. Stochasticity may also push dynamics towards an ‘almost stable’ fix-point, for which the perhaps clearest ecological illustration is provided by laboratory colonies of cyclic populations of the flour beetle *Tribolium castaneum* ; or an ‘almost stable’ chaotic manifold, as is the case of the seasonally-forced chicken-pox model of . The latter phenomenon led to an interesting discussion of the meaning of ‘noise-induced chaos’ .

Predictability: Empirical dynamic modelling
-------------------------------------------

The ‘predictability’ measure used in the previous section is not truly a measure of the level of determinism of the dynamics. Various researchers have proposed to use some form of nonparametric autoregression – sometimes called ‘nonlinear forecasting’ and recently ‘empirical dynamic modeling’ – in combination with leave-one-out cross-validation to quantify predictability in empirical time series. These approaches have used nearest-neighbor methods , kernel regression and local polynomials . The `ntls`-package has implementations of the local polynomial approaches proposed by Tong and co-workers building on the `locfit`-package . The function `ll.order` calculates the cross-validation error across a range of kernel bandwidths and autoregressive lags [3].

We consider the chickenpox SEIR model across a range of stochasticities in transmission and use `ll.order` to calculate the cross-validation predictability of annually strobed versions of the weekly simulations (discarding the first 10 years).

In [ ]:
require(nlts)
llcv=rep(NA,126)
for(i in 1:126){
llfit=ll.order(sqrt(Imat[seq(521,26001, by=52),i]), 
   step=1, order=1:5, bandwidt = seq(0.5, 1.5, 
   by = 0.5), cv=FALSE) 
llcv[i]=min(llfit$grid$GCV)
}
plot(llcv~alpha, ylab="GCV", xlab="Noise variance")

The action of stochastic resonance due to the ‘almost attractor’ is readily visible, as the (generalized) cross-validation error is lowest for intermediate noise variances (fig. \[fig:cXXgcv\]).

In an early application to epidemiology, proposed to use nonparametric prediction-error as a function of prediction lag to distinguish deterministic chaos from noisy limit cycles in measles epidemics. Nonparametric autoregression is a completely ‘mechanism-free’ model for the disease dynamics. We can use the `ll.edm`-function to check that the method produces dynamics that are in rough correspondence to the empirical patterns. Let’s consider a 10 year segment of the 10th weekly simulation. We use order (embedding dimension) 3, because this is indicated as best fit based on the order-consistent estimator (`ll.order`). The resultant empirical dynamic model has roughly appropriate dynamics, though the period is slightly too short (fig. \[fig:cXXedm\])

In [ ]:
x=sqrt(Imat[seq(521,1040, by=1),10])
sim=ll.edm(x=x, order=3, bandwidth=0.8)
plot(x, type="b", ylab="Prevalence", xlab="Week")
lines(sim, col=2)
legend("topright", legend=c("Simulation", "EDM"),
   lty=c(1,1), pch=c(1, NA), col=c("black", "red"))

Appendix: Making a pomp-simulator
---------------------------------

Doing the computations involved in sections \[sec:c10rep\] and \[sec:c10stocres\] are computationally expensive. The `pomp`-package includes a `Csnippet`-function that will compile C code on the fly to speed up calculations. The following provides the C code used in the simulations of the stochastic SEIR model.

In [ ]:
require(pomp)

We first define the Csnippet for the deterministic skeleton of the unobserved process:

In [ ]:
"
double rate[8];  // transition rates
double trans[8];  // transition numbers
double beta=beta0*(1+beta1*cos(2*M_PI*t));//transmission
double lambda = (iota+I*beta)/pop; // FoI \index{Force of infection}

// transition rates
rate[0] = b*pop;     // birth of S
rate[1] = lambda; // infection of S
rate[2] = mu; // death of S
rate[3] = sigma; // latent period of E
rate[4] = mu;  // death of E
rate[5] = gamma;  // recovery of I
rate[6] = mu;  // death of I
rate[7] = mu;      // death of R

// compute the transition numbers
trans[0] = rate[0];
trans[1] = rate[1]*S;
trans[2] = rate[2]*S;
trans[3] = rate[3]*E;
trans[4] = rate[4]*E;
trans[5] = rate[5]*I;
trans[6] = rate[6]*I;
trans[7] = rate[7]*R;

// balance the equations
DS = trans[0] - trans[1] - trans[2];
DE = trans[1] - trans[3] - trans[4];
DI = trans[3] - trans[5] - trans[6];
DR = trans[5] - trans[7];
Dinc = trans[5];  // cumulative recovery
Dpop = trans[0]-trans[2]-trans[4]-trans[6]-trans[7];
" \hlkwb{->} \hlstd{skel}

Then the Csnippet for the stochastic simulator

In [ ]:
"
double rate[8];     // transition rates
double trans[8];    // transition numbers

double beta=beta0*(1+beta1*cos(2*M_PI*t));//transmission
double dW = rgammawn(alpha,dt);  // white noise
double lambda = (iota+I*beta*dW/dt)/pop; 

// transition rates
rate[0] = b*pop;     // birth of S
rate[1] = lambda;     // infection of S
rate[2] = mu;            // death of S
rate[3] = sigma;       //  latent period of E
rate[4] = mu;          // death of E
rate[5] = gamma;     // recovery of I
rate[6] = mu;         // death of I
rate[7] = mu;      // death of R

// compute the transition numbers
trans[0] = rpois(rate[0]*dt);   // births are Poisson
reulermultinom(2, S, &rate[1], dt, &trans[1]);
reulermultinom(2, E, &rate[3], dt, &trans[3]);
reulermultinom(2, I, &rate[5], dt, &trans[5]);
reulermultinom(1, R, &rate[7], dt, &trans[7]);

// balance the equations
S += trans[0] - trans[1] - trans[2];
E += trans[1] - trans[3] - trans[4];
I += trans[3] - trans[5] - trans[6];
R += trans[5] - trans[7];
inc += trans[5];  // cumulative recovery
pop = S + E + I + R;
" \hlkwb{->} \hlstd{rproc}

`pomp` wants Csnippets for the observational process also (even if we only use the object for simulation).

In [ ]:
\hlcom{## Observation model simulator (negative binomial)}
"
double mean = rho*inc;
double size = 1/theta;
reports = rnbinom_mu(size,mean);
" \hlkwb{->} \hlstd{robs}

\hlcom{## Observation model likelihood (negative binomial)}
"
double mean = rho*inc;
double size = 1/theta;
lik = dnbinom_mu(reports,size,mean,give_log);
" \hlkwb{->} \hlstd{dobs}

We need initial conditions

In [ ]:
"
S = nearbyint(pop0*S0);
E = nearbyint(pop0*E0);
I = nearbyint(pop0*I0);
R = nearbyint(pop0*R0);
pop = S+E+I+R;
 inc = 0;
" \hlkwb{->} \hlstd{rinit}

Finally we can build the pomp object. The `dat`-data object defines the times for the stochastic simulation. We are not working with data so the `reports` column is just a dummy.

In [ ]:
dat=data.frame(time=seq(0, 500, by=1/52), reports=NA) 
seirp=pomp(dat, times="time",t0=0,
  rprocess=euler.sim(Csnippet(rproc),delta.t=1/52/20),
  skeleton=vectorfield(Csnippet(skel)),
  rmeasure=Csnippet(robs),
  dmeasure=Csnippet(dobs),
  initializer=Csnippet(rinit),
  params=c(iota=0,beta0=537,beta1=0.3,sigma=36,
      gamma=34.3,alpha=0.015,rho=0.6,theta=1,
       b=0.02,mu=0.02,pop0=5e8,
      S0=0.06,E0=0,I0=0.001,R0=0.939),
  paramnames=c("iota","beta0","beta1","gamma","sigma",
      "alpha","rho","theta", "b","mu","pop0",
      "S0","E0","I0","R0"),
  statenames=c("S","E","I","R","inc","pop"),
  zeronames="inc")

The `pomp`-package has numerous functions to simulate deterministic and stochastic trajectories from `pomp`-objects.

[1] Interestingly paraphrases Henri Poincaré as defining *chance* as sensitive dependence on *unknown* initial conditions as far back as 1908.

[2] Modeling chicken pox, a herpes virus that can reactivate in older individuals in the form of zoster, showed that a the SEIR model can not sustain multiannual (or chaotic) childhood dynamics in the presence of ‘immigration’ of the virus from an adult carrier group.

[3] The method was originally proposed as a nonparametric method to estimate the ‘order’ of a time series .